<font size=6>**TD 2 : classification supervisée**</font>

Julien Velcin, Université Lyon 2 - Master Humanités Numériques

In [1]:
# libraires utilisées

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# différents classifieurs
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
# pour ignorer les avertissements

import warnings
warnings.filterwarnings('ignore')

# chargement des données

Chargement des données à partir d'un fichier .csv (colonnes séparées par une tabulation = "\t")

In [3]:
data_allocine = pd.read_csv("allocine_3k.csv", sep="\t")

In [4]:
data_allocine.shape

(3000, 3)

Affichage d'un extrait :

In [5]:
data_allocine.head()

,id,textes,classes
0,0,un film plus sur les relations amoureuses et l...,0
1,1,Un très bon reportage mais qui pour moi manque...,0
2,2,Excellent .. Un peu d’humour par moment pour u...,1
3,3,La curiosité des dix premières minutes finit p...,0
4,4,Le cinéaste Christophe Gans livre une producti...,1


In [6]:
textes = data_allocine["textes"]
classes = data_allocine["classes"]

Distribution des valeurs pour la classe :

In [7]:
unique_values, counts = np.unique(classes, return_counts=True)

for value, count in zip(unique_values, counts):
    print(f"{value} occurs {count} times")

0 occurs 1496 times
1 occurs 1504 times


# Mise en forme des données

In [8]:
vectorize_allocine = TfidfVectorizer(ngram_range=(1, 2), max_df=0.88, min_df=4)
vectorize_allocine.fit(textes)
data = vectorize_allocine.transform(textes) #.toarray()

In [9]:
subtrain_X, subtest_X, subtrain_y, subtest_y = train_test_split(data, classes,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=123,
                                                   stratify=classes)

In [10]:
subtrain_X.shape

(2100, 14684)

# Apprentissage automatique

## Simple régression logistique

In [11]:
lr = LogisticRegression()

lr.fit(subtrain_X, subtrain_y)
pred_y_lr = lr.predict(subtest_X)

In [12]:
print("Ce qui est prédit : " + str(pred_y_lr[0:20]))

print("La vérité : " + str(subtest_y[0:20]))

Ce qui est prédit : [1 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 0]
La vérité : 95      1
581     0
328     1
596     1
2502    0
1799    0
2331    0
525     1
1526    1
778     1
1370    0
680     0
2402    0
1575    1
70      1
1878    1
1511    1
645     0
1209    0
515     0
Name: classes, dtype: int64


In [13]:
res_test = np.sum(pred_y_lr == subtest_y) / float(len(subtest_y))
                                       
print(f"Réussite en validation (sous-ensemble) {res_test:.1%}")

Réussite en validation (sous-ensemble) 87.1%


A comparer à la réussite sur le jeu d'entraînement :

In [14]:
pred_y_lr = lr.predict(subtrain_X)

res_train = np.sum(pred_y_lr == subtrain_y) / float(len(subtrain_y))
print(f"Réussite en validation (sous-ensemble) {res_train:.1%}")

Réussite en validation (sous-ensemble) 96.9%


Deux remarques importantes à ce stade :

    1. On peut observer une nette différence entre les deux erreurs, signe d'un sur-apprentissage (*overfitting*)
    2. Nous avons utilisé un même nom de variable (pred_y_lr) pour deux résultats différents : c'est une mauvaise habitude et nous allons utiliser 2 variables différentes pour la suite.

Concernant le sur-apprentissage, une piste serait ici de régulariser le modèle avec un terme qui pénalise une trop grande dispersion des poids (régression ridge et lasso).
Essayons par ex. la régression ridge :

In [15]:
lr_ridge = RidgeClassifier(alpha=10.0)
lr_ridge.fit(subtrain_X, subtrain_y)
pred_train_lr_ridge = lr_ridge.predict(subtrain_X)
pred_test_lr_ridge = lr_ridge.predict(subtest_X)

Le paramètre *alpha* indique la force de la régularisation.

In [16]:
res_train = np.sum(pred_train_lr_ridge == subtrain_y) / float(len(subtrain_y))
res_test = np.sum(pred_test_lr_ridge == subtest_y) / float(len(subtest_y))

print(f"Réussite sur :")
print(f"  - ensemble d'entraînement : {res_train:.1%}")
print(f"  - ensemble de test : {res_test:.1%}")

Réussite sur :
  - ensemble d'entraînement : 94.0%
  - ensemble de test : 86.1%


On constate qu'on diminue le sur-apprentissage, mais au prix d'une réussite plus faible...

## Une évaluation plus robuste : la validation croisée

Il s'agit ici d'évaluation la capacité de l'algorithme choisi à résoudre la tâche de classification.
L'objectif est donc plus de choisir l'algorithme que de trouver le modèle lui-même.

In [17]:
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(lr, subtrain_X, subtrain_y, cv=kfold)
print(f"Validation croisée : moyenne {results.mean():.1%} et écart-type {results.std():.2f}")

Validation croisée : moyenne 85.0% et écart-type 0.02


Une fois l'algorithme choisi, il s'agit de réentraîner le modèle sur l'ensemble des données d'entraînement (cf. 3.1).

Pour simplifier le code, on crée une fonction d'affichage des deux erreurs :

In [18]:
def print_error(nom_algo, pred_train, pred_test):
    print(nom_algo + " : ")
    erreur_app = np.sum(pred_train == subtrain_y) / float(len(subtrain_y))
    print(f"  - erreur apparente : {erreur_app:.1%}")
    erreur_gen = np.sum(pred_test == subtest_y) / float(len(subtest_y))
    print(f"  - erreur en généralisation : {erreur_gen:.1%}")                                                          

## Machines à vecteurs supports

In [19]:
svc_lin = SVC(kernel="linear", degree=1)

svc_lin.fit(subtrain_X, subtrain_y)

pred_train_svc_lin = svc_lin.predict(subtrain_X)
pred_test_svc_lin = svc_lin.predict(subtest_X)

print_error("SVC linéaire", pred_train_svc_lin, pred_test_svc_lin)

SVC linéaire : 
  - erreur apparente : 99.0%
  - erreur en généralisation : 87.2%


In [20]:
svc_poly2 = SVC(kernel="poly", degree=2)

svc_poly2.fit(subtrain_X, subtrain_y)

pred_train_svc_poly2 = svc_poly2.predict(subtrain_X)
pred_test_svc_poly2 = svc_poly2.predict(subtest_X)

print_error("SVC poly degré 2", pred_train_svc_poly2, pred_test_svc_poly2)

SVC poly degré 2 : 
  - erreur apparente : 100.0%
  - erreur en généralisation : 86.8%


## Arbres de décision

In [21]:
xgb = XGBClassifier(verbosity=0)

xgb.fit(subtrain_X, subtrain_y)

pred_train_xgb = xgb.predict(subtrain_X)
pred_test_xgb = xgb.predict(subtest_X)

print_error("XGBoost", pred_train_xgb, pred_test_xgb)

XGBoost : 
  - erreur apparente : 99.6%
  - erreur en généralisation : 79.3%


## Réseau de neurones simple

In [22]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3), random_state=1)

mlp.fit(subtrain_X, subtrain_y)

pred_train_mlp = mlp.predict(subtrain_X)
pred_test_mlp = mlp.predict(subtest_X)

print_error("MLP 1 couche cachée", pred_train_mlp, pred_test_mlp)

MLP 1 couche cachée : 
  - erreur apparente : 100.0%
  - erreur en généralisation : 89.2%


# Affichage des résultats

Il faut créer une table qui résumé les principaux résultats obtenus.

In [30]:
#fonction qui retourne les scores d'erreur
def get_error(pred_train, pred_test):
    erreur_app = np.sum(pred_train == subtrain_y) / float(len(subtrain_y))
    erreur_gen = np.sum(pred_test == subtest_y) / float(len(subtest_y))
    return erreur_app, erreur_gen

In [24]:
#Calcul des erreurs

pred_train_lr = lr.predict(subtrain_X)
pred_test_lr = lr.predict(subtest_X)
error_train_lr, error_test_lr = get_error(pred_train_lr, pred_test_lr)
pred_train_lr_ridge = lr_ridge.predict(subtrain_X)
pred_test_lr_ridge = lr_ridge.predict(subtest_X)
error_train_lr_ridge, error_test_lr_ridge = get_error(pred_train_lr_ridge, pred_test_lr_ridge)
error_train_svc_lin, error_test_svc_lin = get_error(pred_train_svc_lin, pred_test_svc_lin)
error_train_svc_poly2, error_test_svc_poly2 = get_error(pred_train_svc_poly2, pred_test_svc_poly2)
error_train_xgb, error_test_xgb = get_error(pred_train_xgb, pred_test_xgb)
error_train_mlp, error_test_mlp = get_error(pred_train_mlp, pred_test_mlp)

In [ ]:
# formatage

In [25]:
noms_algo = ["Régression logistique", "Classification Ridge", "SVM linéaire", "SVM polynôme 2", "XGBoost", "MLP 1 couche cachée"]
erreurs_app = [error_train_lr, error_train_lr_ridge, error_train_svc_lin, error_train_svc_poly2, error_train_xgb, error_train_mlp]
erreurs_gen = [error_test_lr, error_test_lr_ridge, error_test_svc_lin, error_test_svc_poly2, error_test_xgb, error_test_mlp]

In [26]:
res = pd.DataFrame({"algo" : noms_algo, "erreur app": erreurs_app, "erreur gen": erreurs_gen})

In [29]:
res.style.format({
    "erreur app": '{:,.2%}'.format,
    "erreur gen": '{:,.2%}'.format
})

,algo,erreur app,erreur gen
0,Régression logistique,96.90%,87.11%
1,Classification Ridge,93.95%,86.11%
2,SVM linéaire,99.05%,87.22%
3,SVM polynôme 2,100.00%,86.78%
4,XGBoost,99.62%,79.33%
5,MLP 1 couche cachée,100.00%,89.22%
